In [ ]:
!pip install lifelines --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 5.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

from lifelines.statistics import logrank_test
from lifelines import KaplanMeierFitter

In [ ]:
def data_prep(df, topogrup):
    """
    Prepares the dataset for analysis, applying necessary filters and transformations.

    Parameters:
    -----------
    df : pandas.DataFrame
        The dataset to be prepared.
    topogrup : list
        List with the topography group codes to be selected.

    Returns:
    --------
    pandas.DataFrame
        The dataset after the transformations.

    """

    df_aux = df.copy()

    # Colorectal Cancer Topography (C18, C19, C20) - Filter for specific cancer topographies
    df_aux = df_aux[df_aux.TOPOGRUP.isin(topogrup)]

    # Age - Filter for age greater than 19
    df_aux = df_aux[df_aux.IDADE > 19]

    # Residency - Filter for residents of SP
    df_aux = df_aux[df_aux.UFRESID == 'SP']

    # ECGRUP - Remove records with ECGRUP equal to 0, X or Y
    df_aux = df_aux[~df_aux.ECGRUP.isin(['0','X','Y'])]

    # ECGRUP Categorization - Categorize ECGRUP into 'Inicial' and 'Avançado'
    df_aux['ECGRUP_CAT'] = ['Inicial' if ec in ['I', 'II'] else 'Avançado' for ec in df_aux.ECGRUP]

    # ANODIAG - Filter for diagnosis year before 2020 (commented out)
    # df_aux = df_aux[df_aux.ANODIAG < 2020]

    # Microscopic Confirmation - Filter for records with microscopic confirmation (BASEDIAG == 3)
    df_aux = df_aux[df_aux.BASEDIAG == 3]

    # Morphology - Filter for morphology 81403
    df_aux = df_aux[df_aux.MORFO == 81403]

    # TMO and TMOAPOS - Filter for patients who did not undergo TMO and TMOAPOS
    df_aux = df_aux[df_aux.TMO == 0] #TMO - Bone marrow transplantation
    # df_aux = df_aux[df_aux.TMOAPOS == 0] #TMOAPOS - Bone marrow transplantation after

    # Date Columns - Convert to datetime objects
    list_datas = ['DTCONSULT', 'DTDIAG', 'DTTRAT', 'DTULTINFO']

    for col_data in list_datas:
        df_aux[col_data] = pd.to_datetime(df_aux[col_data])

    # Time Calculations - Calculate time differences in days
    df_aux['CONSDIAG'] = (df_aux.DTDIAG - df_aux.DTCONSULT).dt.days #CONSDIAG - Time between consultation and diagnosis
    df_aux['DIAGTRAT'] = (df_aux.DTTRAT - df_aux.DTDIAG).dt.days #DIAGTRAT - Time between diagnosis and treatment
    df_aux['TRATCONS'] = (df_aux.DTTRAT - df_aux.DTCONSULT).dt.days #TRATCONS - Time between treatment and consultation
    df_aux['ULTIDIAG'] = (df_aux.DTULTINFO - df_aux.DTDIAG).dt.days #ULTIDIAG - Time between last information and diagnosis
    df_aux[['DIAGTRAT', 'TRATCONS']] = df_aux[['DIAGTRAT', 'TRATCONS']].fillna(-1) #Fill NaN values with -1

    # Time Categories - Create categorical variables for time differences
    df_aux['CONSDIAG_CAT'] = [0 if consdiag <= 30 else 1 if consdiag <= 60 else 2 for consdiag in df_aux.CONSDIAG]
    df_aux['TRATCONS_CAT'] = [3 if tratcons < 0 else 0 if tratcons <= 60 else 1 if tratcons <= 90 else 2 for tratcons in df_aux.TRATCONS]
    df_aux['DIAGTRAT_CAT'] = [3 if diagtrat < 0 else 0 if diagtrat <= 60 else 1 if diagtrat <= 90 else 2 for diagtrat in df_aux.DIAGTRAT]

    # Metastasis Presence - Create a column indicating the presence of metastasis
    df_aux['PRESENCA_META'] = 0
    df_aux.loc[df_aux.META01.notnull(), 'PRESENCA_META'] = 1

    # Recurrence Presence - Create a variable for presence of recurrence (0 = No, 1 = Yes)
    df_aux['PRESENCA_REC'] = [0 if rec == 1 else 1 for rec in df_aux.RECNENHUM]
    df_aux.loc[(df_aux.PRESENCA_REC == 0) & (df_aux.DTRECIDIVA.notnull()), 'PRESENCA_REC'] = 1

    # DRS - Extract DRS number
    DRS_expand = df_aux.DRS.str.split(' ', expand=True)
    df_aux['DRS'] = DRS_expand[1]

    # DRS_INST - Extract DRS_INST number
    DRS_expand = df_aux.DRS_INST.str.split(' ', expand=True)
    df_aux['DRS_INST'] = DRS_expand[1]

    # Outcome Variables - Create outcome variables (overall survival, cancer-specific survival, and survival at 1, 3, and 5 years)
    df_aux['obito_geral'] = 0
    df_aux['obito_cancer'] = 0

    df_aux['sobrevida_ano1'] = 0
    df_aux['sobrevida_ano3'] = 0
    df_aux['sobrevida_ano5'] = 0

    df_aux.loc[df_aux.ULTINFO > 2, 'obito_geral'] = 1
    df_aux.loc[df_aux.ULTINFO == 3, 'obito_cancer'] = 1

    df_aux.loc[df_aux.ULTIDIAG > 365.25, 'sobrevida_ano1'] = 1
    df_aux.loc[df_aux.ULTIDIAG > 3*365.25, 'sobrevida_ano3'] = 1
    df_aux.loc[df_aux.ULTIDIAG > 5*365.25, 'sobrevida_ano5'] = 1

    # Drop Unused Columns - Remove columns that will not be used in the analysis
    col = df_aux.columns
    drop_cols = ['UFNASC', 'UFRESID', 'CIDADE', 'DTCONSULT', 'CLINICA', 'DTDIAG',
                 'BASEDIAG', 'TOPOGRUP', 'DESCTOPO', 'DESCMORFO', 'T', 'N', 'M',
                 'PT', 'PN', 'PM', 'S', 'G', 'PSA', 'GLEASON', 'LOCALTNM',
                 'IDMITOTIC', 'OUTRACLA', 'META01', 'META02', 'META03', 'META04',
                 'DTTRAT', 'NAOTRAT', 'TRATAMENTO', 'TRATHOSP', 'TRATFANTES',
                 'TRATFAPOS', 'NENHUMANT', 'CIRURANT', 'RADIOANT', 'QUIMIOANT',
                 'HORMOANT', 'TMOANT', 'IMUNOANT', 'OUTROANT', 'DTULTINFO',
                 'CICI', 'CICIGRUP', 'CICISUBGRU', 'FAIXAETAR', 'LATERALI',
                 'INSTORIG', 'PERDASEG', 'ERRO', 'DTRECIDIVA', 'RECNENHUM',
                 'RECLOCAL', 'RECREGIO', 'RECDIST', 'REC01', 'REC02', 'REC03',
                 'REC04', 'CIDO', 'DSCCIDO', 'HABILIT', 'HABIT11', 'HABILIT1',
                 'CIDADEH', 'CIDADE_INS', 'TMO', 'TMOAPOS', 'MORFO']

    col = col.drop(drop_cols)

    return df_aux[col]

# **Data Preparation**

In [ ]:
# Database - Set/2024
!gdown 1aFSW3w4sgOIJdXvUni8Dv_DsIjVGBpRp --quiet

In [ ]:
# Load Data and Display Information
df = pd.read_csv('pacigeral_set24.csv')  # Load the CSV file into a Pandas DataFrame
print(df.shape)  # Print the dimensions of the DataFrame (rows, columns)
df.head(3)  # Display the first 3 rows of the DataFrame

<ipython-input-5-c93721524043>:1: DtypeWarning: Columns (24,25,26,33,37,79,90,91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pacigeral_set24.csv')


(1233793, 105)


,INSTITU,ESCOLARI,IDADE,SEXO,UFNASC,UFRESID,IBGE,CIDADE,CATEATEND,DTCONSULT,...,CIDO,DSCCIDO,HABILIT,HABIT11,HABILIT1,HABILIT2,CIDADEH,DRS_INST,RRAS_INST,CIDADE_INS
0,14,4,49,2,SC,SC,4209409,LAGUNA,9,2011-03-23,...,81603.0,COLANGIOCARCINOMA,14,Inativo,6,5,São Paulo,DRS 01 Grande Sao Paulo,RRAS 06,SAO PAULO
1,8672,9,54,1,SP,SP,3550308,SAO PAULO,9,2006-05-30,...,80903.0,CARCINOMA BASOCELULAR SOE,7,CACON com Serviço de Oncologia Pediátrica,3,2,São Paulo,DRS 01 Grande Sao Paulo,RRAS 06,SAO PAULO
2,19100,9,77,1,SP,SP,3504503,AVARE,9,2003-12-14,...,82113.0,ADENOCARCINOMA TUBULAR,14,Inativo,6,5,Avaré,DRS 06 Bauru,RRAS 09,AVARE


In [ ]:
# Prepare Data for Colorectal Cancer Analysis
df_colo = data_prep(df, ['C18', 'C19', 'C20']) # Prepare the data, filtering for colorectal cancer (ICD-10 code 'C18', 'C19', 'C20')

print(df_colo.shape) # Print the shape (rows, columns) of the filtered DataFrame
df_colo.head(3) # Display the first 3 rows of the filtered DataFrame

(44857, 48)


,INSTITU,ESCOLARI,IDADE,SEXO,IBGE,CATEATEND,DIAGPREV,TOPO,EC,ECGRUP,...,CONSDIAG_CAT,TRATCONS_CAT,DIAGTRAT_CAT,PRESENCA_META,PRESENCA_REC,obito_geral,obito_cancer,sobrevida_ano1,sobrevida_ano3,sobrevida_ano5
15,612374,9,91,2,3550308,2,2,C209,IIA,II,...,0,0,1,0,0,0,0,1,0,0
20,8,9,79,2,3550308,9,1,C180,II,II,...,0,0,0,0,0,1,0,1,1,1
49,8,9,59,2,3534401,9,2,C209,II,II,...,0,0,0,0,1,0,0,1,1,1


In [ ]:
# Final Columns
df_colo.columns

Index(['INSTITU', 'ESCOLARI', 'IDADE', 'SEXO', 'IBGE', 'CATEATEND', 'DIAGPREV',
       'TOPO', 'EC', 'ECGRUP', 'NENHUM', 'CIRURGIA', 'RADIO', 'QUIMIO',
       'HORMONIO', 'IMUNO', 'OUTROS', 'NENHUMAPOS', 'CIRURAPOS', 'RADIOAPOS',
       'QUIMIOAPOS', 'HORMOAPOS', 'IMUNOAPOS', 'OUTROAPOS', 'ULTINFO',
       'CONSDIAG', 'TRATCONS', 'DIAGTRAT', 'ANODIAG', 'DRS', 'RRAS', 'DSCINST',
       'IBGEATEN', 'HABILIT2', 'DRS_INST', 'RRAS_INST', 'ECGRUP_CAT',
       'ULTIDIAG', 'CONSDIAG_CAT', 'TRATCONS_CAT', 'DIAGTRAT_CAT',
       'PRESENCA_META', 'PRESENCA_REC', 'obito_geral', 'obito_cancer',
       'sobrevida_ano1', 'sobrevida_ano3', 'sobrevida_ano5'],
      dtype='object')

In [ ]:
# Calculate and Adjust Survival Time in Months
df_colo['meses_diag'] = np.ceil(df_colo['ULTIDIAG']/30).astype(int)  # Calculate survival time in months, rounding up

df_colo.loc[df_colo.meses_diag == 0, 'meses_diag'] = 1  # Ensure no survival time is zero (set to 1 month)
df_colo.loc[df_colo.meses_diag > 60, ['meses_diag', 'obito_geral']] = [61, 0]  # Cap survival time at 61 months and set obito_geral to 0 for those exceeding the cap.

df_colo.meses_diag.value_counts().sort_index().tail()  # Display the value counts for the last few survival times (for checking)

,count
meses_diag,
57,189
58,183
59,210
60,207
61,12180


**Saving the database into a csv file**

In [ ]:
# Saving database
df_colo.to_csv('colorretal.csv', index=False)